In [1]:
import csv
import requests
import time
import math

In [2]:
# Path to the TSV file
input_file = "Myrold_Biosample_Metadata.csv"
output_file = "Myrold_osm_nearby_features.tsv"

In [3]:
RADIUS = 1000 # meters

In [4]:
# Overpass API endpoint
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

In [5]:
# Function to query OSM for nearby features
def get_nearby_features(lat, lon, radius=500):
    query = f"""
    [out:json];
    (
      node(around:{radius},{lat},{lon});
      way(around:{radius},{lat},{lon});
      relation(around:{radius},{lat},{lon});
    );
    out body;
    """

    response = requests.get(OVERPASS_URL, params={"data": query})
    if response.status_code == 200:
        return response.json().get("elements", [])
    else:
        print(f"OSM API error {response.status_code}")
        return []

In [6]:
# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [7]:
# Function to extract the closest coordinates from an OSM feature
def extract_closest_coordinates(feature, elements_dict, lat, lon):
    min_distance = float("inf")
    closest_lat, closest_lon = None, None

    if "lat" in feature and "lon" in feature:
        return float(feature["lat"]), float(feature["lon"])
    elif "nodes" in feature:
        for node_id in feature["nodes"]:
            if node_id in elements_dict:
                node = elements_dict[node_id]
                node_lat, node_lon = float(node["lat"]), float(node["lon"])
                distance = haversine(lat, lon, node_lat, node_lon)
                if distance < min_distance:
                    min_distance = distance
                    closest_lat, closest_lon = node_lat, node_lon
    elif "members" in feature:
        for member in feature["members"]:
            if "lat" in member and "lon" in member:
                member_lat, member_lon = float(member["lat"]), float(member["lon"])
                distance = haversine(lat, lon, member_lat, member_lon)
                if distance < min_distance:
                    min_distance = distance
                    closest_lat, closest_lon = member_lat, member_lon
            elif member["type"] == "way" and member["ref"] in elements_dict:
                way = elements_dict[member["ref"]]
                if "nodes" in way:
                    for node_id in way["nodes"]:
                        if node_id in elements_dict:
                            node = elements_dict[node_id]
                            node_lat, node_lon = float(node["lat"]), float(node["lon"])
                            distance = haversine(lat, lon, node_lat, node_lon)
                            if distance < min_distance:
                                min_distance = distance
                                closest_lat, closest_lon = node_lat, node_lon

    return closest_lat, closest_lon

In [8]:
# Function to compute characteristic score
def compute_characteristic_score(feature_id, distances):
    total_distance = sum(distances.values())
    if total_distance == 0:
        return {acc: 1.0 for acc in distances}  # If all distances are 0, assign max score
    return {acc: 1 - (dist / total_distance) for acc, dist in distances.items()}

In [9]:
# Function to query OSM for nearby features with retry logic
def get_nearby_features(lat, lon, radius=RADIUS, retries=3):
    query = f"""
    [out:json];
    (
      node(around:{radius},{lat},{lon});
      way(around:{radius},{lat},{lon});
      relation(around:{radius},{lat},{lon});
    );
    out body;
    """

    for attempt in range(retries):
        response = requests.get(OVERPASS_URL, params={"data": query})

        if response.status_code == 200:
            elements = response.json().get("elements", [])
            elements_dict = {el["id"]: el for el in elements if "lat" in el and "lon" in el or "nodes" in el}  # Index nodes and ways
            return elements, elements_dict
        elif response.status_code == 504:
            print(f"OSM API error 504 (Gateway Timeout), retrying {attempt + 1}/{retries}...")
            time.sleep(5)  # Wait before retrying

    print(f"OSM API error 504, skipping this request after {retries} attempts.")
    return [], {}

In [10]:
# Read input CSV into a list of dictionaries
with open(input_file, "r", newline="", encoding="utf-8") as infile:
    reader = csv.DictReader(infile)
    input_data = list(reader)

In [11]:
# Sort input data by accession
input_data.sort(key=lambda x: x.get("accession", ""))

In [12]:
print(f"Last accession after sorting: {input_data[-1]['accession']}")

Last accession after sorting: SAMEA7724228


In [13]:
# Store feature distances for all accessions
distance_map = {}
data = []

# Process each row
for row in input_data:
    accession = row.get("accession", "")
    print(f"Processing accession {accession}...")
    lat = row.get("latitude", "").strip()
    lon = row.get("longitude", "").strip()

    if lat and lon:
        try:
            lat, lon = float(lat), float(lon)
            features, elements_dict = get_nearby_features(lat, lon, RADIUS)

            for feature in features:
                feature_id = feature.get("id", "")
                tags = feature.get("tags", {})
                feature_lat, feature_lon = extract_closest_coordinates(feature, elements_dict, lat, lon)
                distance = ""

                if feature_lat is not None and feature_lon is not None:
                    distance = haversine(lat, lon, feature_lat, feature_lon)
                    if feature_id not in distance_map:
                        distance_map[feature_id] = {}
                    distance_map[feature_id][accession] = distance

                data.append({
                    "accession": accession,
                    "latitude": lat,
                    "longitude": lon,
                    "osm_type": feature.get("type", ""),
                    "osm_id": feature_id,
                    "feature_latitude": feature_lat,
                    "feature_longitude": feature_lon,
                    "distance_meters": distance,
                    "name": tags.get("name", ""),
                    "tags": str(tags),
                    "characteristic_score": 0.0  # Default score to prevent KeyError
                })

        except ValueError:
            print(f"Skipping invalid coordinates for accession {accession}")

        # Sleep to avoid hitting rate limits
        time.sleep(1)

Processing accession SAMEA7724195...
Processing accession SAMEA7724196...
Processing accession SAMEA7724197...
Processing accession SAMEA7724198...
Processing accession SAMEA7724199...
Processing accession SAMEA7724200...
Processing accession SAMEA7724201...
Processing accession SAMEA7724202...
Processing accession SAMEA7724203...
Processing accession SAMEA7724204...
Processing accession SAMEA7724205...
Processing accession SAMEA7724206...
Processing accession SAMEA7724207...
Processing accession SAMEA7724208...
Processing accession SAMEA7724209...
Processing accession SAMEA7724210...
Processing accession SAMEA7724211...
Processing accession SAMEA7724212...
Processing accession SAMEA7724213...
Processing accession SAMEA7724214...
Processing accession SAMEA7724215...
Processing accession SAMEA7724216...
Processing accession SAMEA7724217...
Processing accession SAMEA7724218...
Processing accession SAMEA7724219...
Processing accession SAMEA7724220...
Processing accession SAMEA7724221...
P

In [14]:
# Compute characteristic scores
for feature_id, distances in distance_map.items():
    scores = compute_characteristic_score(feature_id, distances)
    for row in data:
        if row["osm_id"] == feature_id:
            row["characteristic_score"] = scores.get(row["accession"], 0.0)  # Use get() to prevent KeyError

In [17]:
# Rank features by characteristic score for each accession
data.sort(key=lambda x: (x["accession"], -x["characteristic_score"]))

In [18]:
# Assign rank within each accession
temp_accession = None
rank = 0
for row in data:
    if row["accession"] != temp_accession:
        temp_accession = row["accession"]
        rank = 1
    row["rank"] = rank
    rank += 1

In [19]:
# Write results to TSV, filtering out empty metadata
fieldnames = ["accession", "latitude", "longitude", "osm_type", "osm_id", "feature_latitude", "feature_longitude", "distance_meters", "name", "tags", "characteristic_score", "rank"]
with open(output_file, "w", newline="", encoding="utf-8") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter="\t")
    writer.writeheader()
    writer.writerows(row for row in data if row["tags"] != "{}")